# Analyze Forced Photometry in ComCam Data

In [1]:
# %pip install lsdb dask nested-dask 

In [2]:
import lsdb
lsdb.__version__

'0.5.0'

In [3]:
from pathlib import Path

release = 'w_2025_07'
hats_path = Path("/sdf/data/rubin/shared/lsdb_commissioning/hats/") / release
# list dir
print(list(map(str, hats_path.iterdir())))

comcam_obj = hats_path / "object"
comcam_src = hats_path / "forcedSource"

['/sdf/data/rubin/shared/lsdb_commissioning/hats/w_2025_07/diaSource', '/sdf/data/rubin/shared/lsdb_commissioning/hats/w_2025_07/object_lc_x_ztf_dr22', '/sdf/data/rubin/shared/lsdb_commissioning/hats/w_2025_07/source', '/sdf/data/rubin/shared/lsdb_commissioning/hats/w_2025_07/object_lc_x_ps1', '/sdf/data/rubin/shared/lsdb_commissioning/hats/w_2025_07/diaObject_lc_x_ztf_dr22', '/sdf/data/rubin/shared/lsdb_commissioning/hats/w_2025_07/diaObject_lc_x_ps1', '/sdf/data/rubin/shared/lsdb_commissioning/hats/w_2025_07/object_lc', '/sdf/data/rubin/shared/lsdb_commissioning/hats/w_2025_07/diaForcedSource', '/sdf/data/rubin/shared/lsdb_commissioning/hats/w_2025_07/object', '/sdf/data/rubin/shared/lsdb_commissioning/hats/w_2025_07/diaObject_lc', '/sdf/data/rubin/shared/lsdb_commissioning/hats/w_2025_07/diaObject', '/sdf/data/rubin/shared/lsdb_commissioning/hats/w_2025_07/forcedSource']


## Start Dask client

In [4]:
from dask.distributed import Client

# Start with a small client
client = Client(n_workers=1, memory_limit="16GB", threads_per_worker=1)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 1
Total threads: 1,Total memory: 14.90 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:5587,Workers: 1
Dashboard: http://127.0.0.1:8787/status,Total threads: 1
Started: Just now,Total memory: 14.90 GiB
Comm: tcp://127.0.0.1:28709,Total threads: 1
Dashboard: http://127.0.0.1:19675/status,Memory: 14.90 GiB
Nanny: tcp://127.0.0.1:32667,


# Load GAIA Data Around the COMCAM Field

In [5]:
source_lsdb_server =  "http://epyc.astro.washington.edu:43210/hats" # https://data.lsdb.io/hats is the alternative server

In [46]:
gaia_columns = [
    "solution_id",
    "designation",
    "source_id",
    "random_index",
    "ref_epoch",
    "ra",
    "ra_error", 
    "dec",
    "dec_error",
    "ruwe",
    "phot_variable_flag",
    "phot_g_mean_flux_over_error",
    "phot_bp_mean_flux_over_error",
    "phot_rp_mean_flux_over_error",
]
gaia_columns

['solution_id',
 'designation',
 'source_id',
 'random_index',
 'ref_epoch',
 'ra',
 'ra_error',
 'dec',
 'dec_error',
 'ruwe',
 'phot_variable_flag',
 'phot_g_mean_flux_over_error',
 'phot_bp_mean_flux_over_error',
 'phot_rp_mean_flux_over_error']

In [ ]:
from upath import UPath
import hats
from lsdb.core.search import ConeSearch

catalogs_dir = UPath(source_lsdb_server)

# Gaia
gaia_path = catalogs_dir / "gaia_dr3" / "gaia"

# Define a 0.7 degree cone region of interest
# This includes the so called `Fornax dSph` field, one of the ComCam fields from
# https://community.lsst.org/t/locations-of-target-fields-observed-during-on-sky-commissioning-campaign-with-comcam/9609
cone_search = ConeSearch(ra=40, dec=-34.45, radius_arcsec=0.7 * 3600)


hats_gaia = hats.read_hats(gaia_path)

#gaia = lsdb.read_hats(gaia_path, columns=hats_gaia.schema.names, search_filter=cone_search) #filters=gaia_filters, )
gaia = lsdb.read_hats(gaia_path, columns=gaia_columns, search_filter=cone_search) #filters=gaia_filters, )

In [ ]:
# Filter column phot_variable_flag=CONSTANT
"""
gaia_filters = [
    #["phot_variable_flag", "=", "'CONSTANT'"],
    ["phot_g_mean_flux_over_error", ">", 20],
    ["phot_bp_mean_flux_over_error", ">", 20],
    ["phot_rp_mean_flux_over_error", ">", 20],
    #["ruwe", "<", 1.2],
]
"""
filtered_gaia = gaia.query("ruwe < 1.2").query("phot_variable_flag == 'CONSTANT'")#.query("phot_rp_mean_flux_over_error > 20")

In [ ]:
filtered_gaia_compute = filtered_gaia.head(100)
filtered_gaia_compute

In [64]:
filtered_gaia_compute.query("phot_variable_flag == 'VARIABLE'").head(10)

,solution_id,designation,source_id,random_index,ref_epoch,ra,ra_error,dec,dec_error,ruwe,phot_variable_flag,phot_g_mean_flux_over_error,phot_bp_mean_flux_over_error,phot_rp_mean_flux_over_error
_healpix_29,,,,,,,,,,,,,,
2525040430023494061,1636148068921376768,Gaia DR3 5050080859409400704,5050080859409400704,1354361632,2016.0,39.977873,0.046924,-35.096808,0.061815,1.082952,VARIABLE,522.8178,57.55541,133.74568
2525041624732325976,1636148068921376768,Gaia DR3 5050083230231341312,5050083230231341312,283775503,2016.0,40.059224,0.024904,-34.974525,0.033771,1.043589,VARIABLE,333.2393,72.04738,86.01284
2525080610398506618,1636148068921376768,Gaia DR3 5050161188181670400,5050161188181670400,1336335345,2016.0,40.451691,0.071166,-35.026323,0.100079,1.020469,VARIABLE,394.31287,54.47336,51.219162


# Find GAIA objects that exhibit very little variability. 
For instance standard deviation less than 0.05 mag (for instance, flux_over_error >20, ruwe<1.2, phot_variable_flag=CONSTANT). 

## Loading & Nesting Forced Sources

In [ ]:
# Load the Forced Source + MJD Table
from lsdb import read_hats


#BRIGHTEST_R_MAG = 21.5


obj = read_hats(
    comcam_obj,
    columns=["objectId", "coord_ra", "coord_dec"],
    #filters=[("r_psfMag", ">", BRIGHTEST_R_MAG)],
)
src_flat = read_hats(
    comcam_src,
    columns=[
        "objectId", 
        "coord_ra", "coord_dec",
        "band",
        "midpointMjdTai",
        "psfFlux", "psfFluxErr", "psfFlux_flag",
        "psfMag", "psfMagErr",
        "pixelFlags_suspect", "pixelFlags_saturated", "pixelFlags_cr", "pixelFlags_bad",
        "forcedSourceId",
        "detector",
        "visit",
    ],
)
src_nested = obj.join_nested(
    src_flat,
    nested_column_name="lc",
    left_on="objectId",
    right_on="objectId",
)
src_nested

# Crossmatch ComCam Data and Gaia

# Verify Crossmatch

Verify that the crossmatch is correct, for instance by confirming that objects that you have found in ComCam has roughly the same brightness in magnitudes as reported in GAIA. For example, compare Gaia RP mag to Rubin’s r mag, allowing up to 1-mag difference.

# Apply Forced Photometry
Measure standard deviation of the points using forced photometry and forced photomery on difference images. 

# Compare your measurements with the reported errors

